Alle cleanen Datasets importieren

In [ ]:
import pandas as pd

arr = pd.read_csv("../data/processed/clean_FZAG_DATA_arrivals.csv")
dep = pd.read_csv("../data/processed/clean_FZAG_DATA_departures.csv")
metar = pd.read_csv("../data/processed/clean_metar_lszh_2023-2025.csv")


Alle Spalten, die eine Zeit als String angegeben haben umwandeln zu Datetime

In [ ]:
arr['actual_in_block_time_utc'] = pd.to_datetime(arr['actual_in_block_time_utc'], utc=True)
arr['scheduled_in_block_time_utc'] = pd.to_datetime(arr['scheduled_in_block_time_utc'], utc=True)

dep['actual_off_block_time_utc'] = pd.to_datetime(dep['actual_off_block_time_utc'], utc=True)
dep['scheduled_off_block_time_utc'] = pd.to_datetime(dep['scheduled_off_block_time_utc'], utc=True)

metar['time'] = pd.to_datetime(metar['time'], utc=True)

30' Zeitintervall für den Zeitraum der Datasets generieren -> manuell gesetzte Grenzen vom 01.01.23, 00:00 UTC - 27.09.25 00:00 UTC

In [ ]:
#Zeitgrenzen definieren
start = pd.Timestamp("2023-01-01 00:00:00", tz="UTC")
end = pd.Timestamp("2025-09-27 00:00:00", tz="UTC")

#30' Zeitintervall generieren
time_index = pd.date_range(start=start, end=end, freq='30min', inclusive='left') #inclusive left to include start time and exclude end time

#Dataframe mit Zeitintervall erstellen
time_windows = pd.DataFrame({'start_time': time_index})
time_windows['end_time'] = time_windows['start_time'] + pd.Timedelta(minutes=30)

'''
Zum überprüfen, ob Dataframe korrekt erstellt wurde

print(len(time_windows))
print(time_windows.head())
print(time_windows.tail())
'''

Feature-Dataframe erstellen mit time_windows als Basis -> Das wird das finale Dataframe, wo alle Features fürs Modell eingebaut sind

In [ ]:
features_df = time_windows.copy()
#print(features_df.head())

Helper Dataframe erstellen -> Hier können neue Features eingefügt werden, die nur zur Unterstützung da sind und nicht im finalen "features_df" landen

In [ ]:
helper_df = time_windows.copy()

Temporale Komponenten extrahieren und zu helper_df hinzufügen

In [ ]:
# Extract from start_time 
helper_df['month'] = helper_df['start_time'].dt.month - 1 #January=0, December=11
helper_df['day_of_week'] = helper_df['start_time'].dt.dayofweek #Monday=0, Sunday=6
helper_df['minute_of_day'] = (
        helper_df['start_time'].dt.hour * 60 +
        helper_df['start_time'].dt.minute
    )  # 0–1439
#helper_df


Hilfsfunktion für zyklische Encodierung erstellen

In [ ]:
import numpy as np

def sin_cos(x,T):
    return np.sin(2 * np.pi * x / T), np.cos(2 * np.pi * x / T)


'''
T = Periodenlänge, z.B. 12 für Monate, 7 für Wochentage, 1440 für Minuten am Tag
x = Wert, der umgewandelt werden soll, z.B. Monat 0-11, Wochentag 0-6, Minute 0-1439 (Schematisch: x ist Element von [0, T-1])
'''

Temporale Komponenten zyklisch encodieren und zu helper_df hinzufügen

In [ ]:
helper_df['month_sin'], helper_df['month_cos'] = sin_cos(helper_df['month'], 12)
helper_df['day_of_week_sin'], helper_df['day_of_week_cos'] = sin_cos(helper_df['day_of_week'], 7)
helper_df['minute_of_day_sin'], helper_df['minute_of_day_cos'] = sin_cos(helper_df['minute_of_day'], 1440)

#helper_df

Zyklisch encodierte temporale Komponenten zum features_df hinzufügen

In [ ]:
# Select only the cyclic columns
cyclic_cols = [
    'start_time',
    'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos',
    'minute_of_day_sin', 'minute_of_day_cos'
]

# Merge them into existing features_df
features_df = features_df.merge(helper_df[cyclic_cols], on='start_time', how='left')


METAR Komponenten extrahieren

In [ ]:
metar_use = (
    metar.rename(columns={
        "time": "metar_time_utc",
        "wind_dir_cleaned": "wind_dir_deg",
        "wind_speed_cleaned": "wind_speed_kt",
        "vis_cleaned": "visibility_m",
        "temp_cleaned": "temperature_c",
        "dewpt_cleaned": "dewpoint_c",
        "press_cleaned": "qnh_hpa",
        "is_wind_variable": "is_wind_variable"
    })
    # keep only the columns we actually want
    [["metar_time_utc", "wind_dir_deg", "wind_speed_kt",
      "visibility_m", "temperature_c", "dewpoint_c",
      "qnh_hpa", "is_wind_variable"]]
    .copy()
)

# ensure is_wind_variable is boolean True/False
metar_use["is_wind_variable"] = metar_use["is_wind_variable"].astype(bool)


Timestamps von helper_df und metar_use sortieren

In [ ]:
# Make sure both are sorted
helper_df   = helper_df.sort_values("start_time").copy()
metar_use   = metar_use.sort_values("metar_time_utc").copy()

Winddirection zyklisch encodieren und Condition erstellen, was passiert, wenn Wind VRB ist

In [ ]:
rad = np.deg2rad(metar_use['wind_dir_deg'])
metar_use['wind_dir_sin'] = np.sin(rad)
metar_use['wind_dir_cos'] = np.cos(rad)

if "is_wind_variable" in metar_use.columns:
    mask_var = metar_use["is_wind_variable"].astype(bool)
    metar_use.loc[mask_var, ["wind_dir_sin", "wind_dir_cos"]] = np.nan

    #Falls Wind variabel ist, setze wind_dir_sin und wind_dir_cos auf NaN

Visibility vorbereiten, um danach kategorisiert werden zu können

In [ ]:
# 1) Cap all values at 10,000 m
#    METAR reports "99999" for "10 km or more", not literally 99999 m.
metar_use['visibility_m_clean'] = metar_use['visibility_m'].clip(upper=10000.0)

# 2) Create a boolean flag for "10 km or more"
#    Preserves the info that visibility exceeded the METAR upper limit.
metar_use['vis_ge_10km'] = metar_use['visibility_m'] >= 10000

Visibility kategorisieren

In [ ]:
bins   = [0, 500, 1000, 2000, 5000, 8000, 10000]
labels = ['≤500m', '500–1000m', '1–2km', '2–5km', '5–8km', '8–10km']
metar_use['vis_category'] = pd.cut(
    metar_use['visibility_m_clean'],
    bins=bins, labels=labels, include_lowest=True, right=True
).astype('category')

Spread (Temperatur - Taupunkt) berechnen

In [ ]:
metar_use['spread_c'] = metar_use['temperature_c'] - metar_use['dewpoint_c']

METAR Komponenten zu helper_df hinzufügen

In [ ]:
cols_to_bring = [
    'metar_time_utc',
    'wind_dir_deg', 'wind_speed_kt', 'is_wind_variable',
    'wind_dir_sin', 'wind_dir_cos',
    'visibility_m_clean', 'vis_ge_10km', 'vis_category',
    'temperature_c', 'dewpoint_c', 'spread_c', 'qnh_hpa'
]

helper_df = pd.merge_asof(
    helper_df,
    metar_use[cols_to_bring],
    left_on='start_time',
    right_on='metar_time_utc',
    direction='backward'
)


METAR Komponenten zu features_df hinzufügen

In [ ]:
features_df = features_df.merge(
    helper_df[[
        'start_time',
        'wind_dir_sin', 'wind_dir_cos',
        'wind_speed_kt', 'is_wind_variable',
        'vis_ge_10km', 'vis_category',
        'temperature_c', 'spread_c', 'qnh_hpa'
    ]],
    on='start_time',
    how='left'
)

Ferien extrahieren (National + ZH spezifisch) und zu helper_df hinzufügen

In [ ]:
import holidays

helper_df['start_time_local'] = helper_df['start_time'].dt.tz_convert('Europe/Zurich')

# 2) Local date in Zurich -> Nötig, da Ferien jeweils in LT angegeben sind und nicht in UTC
helper_df['start_time_local'] = helper_df['start_time'].dt.tz_convert('Europe/Zurich') 
helper_df['date_local'] = helper_df['start_time_local'].dt.date

# 3) Build Zurich holidays for the years present
years_needed = sorted(helper_df['start_time_local'].dt.year.unique().tolist())
ch_holidays_zh = holidays.Switzerland(prov='ZH', years=years_needed)

# 4) Use a SET of dates for membership checks (critical!)
holiday_dates = set(ch_holidays_zh.keys())

helper_df['is_holiday'] = helper_df['date_local'].isin(holiday_dates)

Ferien zum features_df hinzufügen

In [ ]:
features_df = features_df.merge(
    helper_df[['start_time', 'is_holiday']],
    on='start_time',
    how='left'
)

Anzahl Scheduled Departures & Arrivals für 30' Intervalle aufsummieren

In [ ]:
#Scheduled departures per 30' interval
DEP_TIME_COL = "scheduled_off_block_time_utc"     
dep_sched = (
    dep.assign(slot_start = dep[DEP_TIME_COL].dt.floor("30min"))
       .groupby("slot_start", as_index=False)
       .size()
       .rename(columns={"size": "scheduled_departures"})
)

# b) Scheduled arrivals per 30' interval
ARR_TIME_COL = "scheduled_in_block_time_utc"      
arr_sched = (
    arr.assign(slot_start = arr[ARR_TIME_COL].dt.floor("30min"))
       .groupby("slot_start", as_index=False)
       .size()
       .rename(columns={"size": "scheduled_arrivals"})
)

Anzahl Scheduled Departures & Arrivals zum helper_df hinzufügen

In [ ]:
# --- Merge departures into helper_df ---
helper_df = (
    helper_df
    .merge(dep_sched, left_on="start_time", right_on="slot_start", how="left")
    .drop(columns=["slot_start"])
)

# --- Merge arrivals into helper_df ---
helper_df = (
    helper_df
    .merge(arr_sched, left_on="start_time", right_on="slot_start", how="left")
    .drop(columns=["slot_start"])
)

# --- Fill missing values (slots with no traffic) ---
helper_df[["scheduled_departures", "scheduled_arrivals"]] = (
    helper_df[["scheduled_departures", "scheduled_arrivals"]]
    .fillna(0)
    .astype(int)
)

Anzahl Scheduled Departures & Arrivals zum Features df hinzufügen

In [ ]:
# Bring just the relevant columns into features_df
features_df = features_df.merge(
    helper_df[["start_time", "scheduled_departures", "scheduled_arrivals"]],
    on="start_time",
    how="left"
)

# Optional safety: fill NaNs (in case of unmatched times)
features_df[["scheduled_departures", "scheduled_arrivals"]] = (
    features_df[["scheduled_departures", "scheduled_arrivals"]]
    .fillna(0)
    .astype(int)
)

Departure Delay berechnen -> Annahme: Departure Delay = Differenz zw. actual und scheduled off block time

In [ ]:
dep['delay_departure_minutes'] = (dep['actual_off_block_time_utc'] - dep['scheduled_off_block_time_utc']).dt.total_seconds() / 60.0

Average Departure Delay berechnen pro 30' Intervall berechnen

In [ ]:
DEP_ACT_TIME_COL = "scheduled_off_block_time_utc"   
DELAY_COL        = "delay_departure_minutes"        

# 1) Compute per-slot average actual departure delay (for the slot itself)
dep_delay_slot = (
    dep.assign(slot_start = dep[DEP_ACT_TIME_COL].dt.floor("30min"))
       .groupby("slot_start", as_index=False)[DELAY_COL]
       .mean()
       .rename(columns={DELAY_COL: "avg_dep_delay_slot_minutes"})
)

Average Departure Delay zu helper_df hinzufügen

In [ ]:
# Ensure time order for safe merging
helper_df = helper_df.sort_values("start_time").copy()
dep_delay_slot = dep_delay_slot.sort_values("slot_start").copy()

# Add avg delay of the *current* slot into helper_df
helper_df = (
    helper_df
    .merge(dep_delay_slot, left_on="start_time", right_on="slot_start", how="left")
    .drop(columns=["slot_start"])
)

# Fill slots without departures with 0
helper_df["avg_dep_delay_slot_minutes"] = helper_df["avg_dep_delay_slot_minutes"].fillna(0).astype(float)

Average Departure Delay vom vorherigem Slot und von den letzten 6 Stunden zu helper_df hinzufügen

In [ ]:
# Sort again to be explicit
helper_df = helper_df.sort_values("start_time").copy()

# Previous slot average (lag 1)
helper_df["avg_dep_delay_prev_slot_minutes"] = (
    helper_df["avg_dep_delay_slot_minutes"].shift(1).fillna(0)
)

# Rolling average over the past 6h (12 slots) — exclude current slot via shift
helper_df["avg_dep_delay_past_6h_minutes"] = (
    helper_df["avg_dep_delay_slot_minutes"]
    .shift(1)                           # exclude current slot
    .rolling(window=12, min_periods=1)  # 12 * 30min = 6h
    .mean()
    .fillna(0)
)

Average Departure Delay vom aktuellen und vorherigem Slot + Average Departure Delay der letzten 6 Stunden zum features_df hinzufügen

In [ ]:
cols_to_bring = [
    "start_time",
    "avg_dep_delay_slot_minutes",
    "avg_dep_delay_prev_slot_minutes",
    "avg_dep_delay_past_6h_minutes",
]

features_df = features_df.merge(
    helper_df[cols_to_bring],
    on="start_time",
    how="left"
)


Anzahl Scheduled Departures pro Flugzeugkategorie (B-F) pro 30' Slot extrahieren und zu helper_df hinzufügen

In [ ]:
# --- Define key columns ---
DEP_ACT_TIME_COL = "scheduled_off_block_time_utc"
CODE_COL         = "aircraft_category_icao_code"

# 1) Assign departures to 30-minute slots and normalize code letters
dep_slots = (
    dep.assign(
        slot_start  = dep[DEP_ACT_TIME_COL].dt.floor("30min"),
        code_letter = dep[CODE_COL].astype(str).str.upper().str.strip()
    )
)

# 2) Keep valid ICAO letters (B–F)
valid_letters = list("BCDEF")
dep_slots = dep_slots[dep_slots["code_letter"].isin(valid_letters)]

# 3) Count departures per slot and aircraft category
counts_long = (
    dep_slots.groupby(["slot_start", "code_letter"], as_index=False)
             .size()
             .rename(columns={"size": "count"})
)

# 4) Pivot: scheduled_B … scheduled_F
counts_wide = (
    counts_long.pivot(index="slot_start", columns="code_letter", values="count")
               .reindex(columns=valid_letters, fill_value=0)
               .add_prefix("scheduled_")
               .reset_index()
)

# 5) Merge into helper_df
helper_df = (
    helper_df.merge(counts_wide, left_on="start_time", right_on="slot_start", how="left")
             .drop(columns=["slot_start"])
)

# 6) Replace NaN (slots with no departures) with 0
scheduled_cols = [f"scheduled_{c}" for c in valid_letters]
helper_df[scheduled_cols] = helper_df[scheduled_cols].fillna(0).astype("int64")



Anzahl Scheduled Departures pro Flugzeugkategorie (B-F) zu features_df hinzufügen

In [ ]:
# Columns to bring over from helper_df
valid_letters  = list("BCDEF")
scheduled_cols = [f"scheduled_{c}" for c in valid_letters]

# Merge into features_df via start_time
features_df = features_df.merge(
    helper_df[['start_time'] + scheduled_cols],
    on='start_time',
    how='left'
)

# Ensure zeros for slots with no departures and stable dtype
features_df[scheduled_cols] = features_df[scheduled_cols].fillna(0).astype('int64')


Überprüfen, welche RWY Concepts im Dataset überhaupt existieren

In [ ]:
print(sorted(dep['runway_concept'].dropna().unique().tolist()))

Zählen, wie oft welches RWY Concept im Dataset gebraucht wurde

In [ ]:
# Assuming your feature column is called 'scheduled_rwy_concept'
concept_counts = dep['runway_concept'].value_counts().sort_values(ascending=False)
(dep["runway_concept"]
 .value_counts(normalize=True)
 .mul(100)
 .round(5)
)



Actual und Scheduled Runway Concept extrahieren und zu helper_df hinzufügen

In [ ]:
'''
Das Runway Concept ist im Dataset pro Flug angegeben.
Hier wird das häufigste Runway Concept pro 30' Slot extrahiert und zu helper_df hinzugefügt.
Scheduled Concept ist der Wert des vorherigen Slots. 
Ist kein Konzept hinterlegt, wird NaN gesetzt.
Da einige Concepts <0.1% des gesamten Datasets ausmachen, werden diese als neues Konzept "other_concept" zusammengefasst
'''

rwy_slot = (
    dep.assign(slot_start = dep['scheduled_off_block_time_utc'].dt.floor('30min'))
       .groupby('slot_start', as_index=False)['runway_concept']
       .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
       .rename(columns={'runway_concept': 'actual_rwy_concept'})
)

helper_df = (
    helper_df.merge(rwy_slot, left_on='start_time', right_on='slot_start', how='left')
             .drop(columns=['slot_start'])
)

# Make it categorical (important for XGBoost)
helper_df['actual_rwy_concept'] = helper_df['actual_rwy_concept'].astype('category')

helper_df = helper_df.sort_values('start_time').copy()
helper_df['scheduled_rwy_concept'] = helper_df['actual_rwy_concept'].shift(1)
helper_df['scheduled_rwy_concept'] = helper_df['scheduled_rwy_concept'].astype('category')

#Seltene Konzepte (<1%) finden
freq = helper_df['actual_rwy_concept'].value_counts(normalize=True, dropna=True)
rare_cats = freq[freq < 0.01].index  # <1 %

# 5) Seltene Konzepte zu "other_concept" mappen (für beide Spalten konsistent)
for col in ['actual_rwy_concept', 'scheduled_rwy_concept']:
    # Kategorie "other_concept" hinzufügen
    helper_df[col] = helper_df[col].cat.add_categories(['other_concept'])
    # seltene Kategorien ersetzen
    helper_df.loc[helper_df[col].isin(rare_cats), col] = 'other_concept'
    # ungenutzte Kategorien aufräumen
    helper_df[col] = helper_df[col].cat.remove_unused_categories()

Scheduled Runway Concept zu features_df hinzufügen

In [ ]:
features_df = features_df.merge(
    helper_df[['start_time', 'scheduled_rwy_concept']],
    on='start_time',
    how='left'
)

Verhältnis von Departures und Arrivals ausrechnen und zu helper_df hinzufügen

In [ ]:
helper_df["arr_dep_ratio"] = (
    helper_df["scheduled_arrivals"]
    / helper_df["scheduled_departures"].replace(0, np.nan)
)

# Replace divisions by 0 and NaNs (no traffic) with 0
helper_df["arr_dep_ratio"] = helper_df["arr_dep_ratio"].fillna(0)


Verhältnis von Departures und Arrivals zu features_df hinzufügen

In [ ]:
features_df = features_df.merge(
    helper_df[["start_time", "arr_dep_ratio"]],
    on="start_time",
    how="left"
)

features_df["arr_dep_ratio"] = features_df["arr_dep_ratio"].fillna(0)


In [ ]:
helper_df

In [ ]:
features_df

features_df und helper_df als Pickle-File exportieren in den Ordner data/processed. Anders als bei CSV gehen hier Datatypes nicht verloren

In [ ]:
output_path_f = "../data/processed/features.pkl"
output_path_h = "../data/processed/helper.pkl"

#Save cleaned dataframe to pkl
features_df.to_pickle(output_path_f)
helper_df.to_pickle(output_path_h)

print(f" Cleaned file saved to: {output_path_f}")
print(f" Cleaned file saved to: {output_path_h}")